In [5]:
import requests

In [6]:
from bs4 import BeautifulSoup

In [7]:
locations = 'Lào Cai, Yên Bái, Điện Biên, Hòa Bình, Lai Châu, Sơn La, Hà Giang, Cao Bằng, Bắc Kạn, Lạng Sơn, Tuyên Quang, Thái Nguyên, Phú Thọ, Bắc Giang, Quảng Ninh, Bắc Ninh, Hà Nam, Hà Nội, Hải Dương, Hưng Yên, Hải Phòng, Nam Định, Ninh Bình, Thái Bình, Vĩnh Phúc, Thanh Hoá , Nghệ An'

In [8]:
import re
import sys

patterns = {
    '[àáảãạăắằẵặẳâầấậẫẩ]': 'a',
    '[đ]': 'd',
    '[èéẻẽẹêềếểễệ]': 'e',
    '[ìíỉĩị]': 'i',
    '[òóỏõọôồốổỗộơờớởỡợ]': 'o',
    '[ùúủũụưừứửữự]': 'u',
    '[ỳýỷỹỵ]': 'y'
}

def convert(text):
    """
    Convert from 'Tieng Viet co dau' thanh 'Tieng Viet khong dau'
    text: input string to be converted
    Return: string converted
    """
    output = text
    for regex, replace in patterns.items():
        output = re.sub(regex, replace, output)
        # deal with upper case
        output = re.sub(regex.upper(), replace.upper(), output)
    return output

In [9]:
citys = convert(locations)
citys = [i.lower().strip() for i in citys.split(", ")]
citys=['/' + i.replace(" ",'-') for i in citys]
citys2ids = requests.get("https://thongtindoanhnghiep.co/api/city").json()['LtsItem']
citys2ids = [i for i in citys2ids if i['SolrID'] in citys]

In [10]:
districts = []
for city in citys2ids:
    city_id = city['ID']
    url = f'https://thongtindoanhnghiep.co/api/city/{city_id}/district'
    r = requests.get(url)
    districts += r.json()
    
districts = [i['SolrID'][1:] for i in districts]

In [ ]:
num_records_by_districts = {}
e = 0
for i in districts:
    e+=1
    if(e%20) == 0:
        print(e)
    url = f'https://thongtindoanhnghiep.co/api/company?l={i}'
    r = requests.get(url).json()
    num_records_by_districts[i] = r["Option"]['TotalRow']
    

In [ ]:
import pickle


In [14]:
def get_closed_date(tax_code):
    url = f'https://thongtindoanhnghiep.co/api/company/{tax_code}'
    r = requests.get(url).json()
    closed_date = r['NgayDongMST']
    return closed_date


In [1]:
import time

In [2]:
def parse_districts(district):
    num_records = num_records_by_districts[district]
    num_pages = num_records//100 + 2
    for page in range(1,num_pages):
        time.sleep(0.1)
        url = f'https://thongtindoanhnghiep.co/api/company?r=100&p={page}&l={district}'
        print(district, page)
        try:
            r = requests.get(url).json()['LtsItems']
            res = [{'company_name': i['Title'], 
                      'location_detail': i['DiaChiCongTy'],
                      'tax_code': i['MaSoThue'],
                      'established_date': i['NgayCap'],
                      'sector': i['NganhNgheTitle'],
                      'province': i['TinhThanhTitle']
                     } for i in r]
            df = pd.DataFrame(res, columns=['company_name','province','location_detail', 'sector', 'tax_code', 'established_date'])
            fn = district.split("/")[0] + '.csv'
            with open(fn, 'a') as f:
                df.to_csv(f, index = False, header = False, sep='\t')
        except:
            print('error', url)
            with open("error.txt",'a+') as f:
                f.write(url)
    print('DONE', district, num_records)

In [3]:
from multiprocessing import Pool

In [4]:
p = Pool(8)
p.map(parse_districts,districts )

NameError: name 'districts' is not defined